In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset,random_split
import torch.nn.functional as F
import time
import os
import re
from itertools import chain
from transformers import BertTokenizer, BertForMaskedLM
from IPython.display import clear_output
import pandas as pd

PRETRAINED_MODEL_NAME = "bert-base-chinese" #英文pretrain(不區分大小寫)
print(torch.__version__)

/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu-gpu/anaconda3/e

1.1.0


In [2]:
# get pre-train tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
vocab = tokenizer.vocab
print("dict size", len(vocab))

# see some token and index mapping
import random
random_tokens = random.sample(list(vocab), 10)
random_ids = [vocab[t] for t in random_tokens]

print("{0:20}{1:15}".format("token", "index"))
print("-" * 25)
for t, id in zip(random_tokens, random_ids): #隨便看幾個字
    print("{0:15}{1:10}".format(t, id))

dict size 21128
token               index          
-------------------------
adam                11194
##輓                 19793
##硅                 17852
cut                 12322
##≧                 13548
##reen              10902
癒                    4618
[unused3]               3
待                    2521
e                     147


In [3]:
train = pd.read_csv('train.csv')
val = pd.read_csv('validation.csv')
total = pd.concat([train, val])
total = total.reset_index(drop=True)
total_0 = total[total["Label"] == 0]
total_1 = total[total["Label"] == 1]

In [4]:
# Label_1
num_1 = total_1.shape[0]
random_1 = random.sample(range(num_1), num_1)
train_1 = total_1.iloc[random_1[:108]]
test_1 = total_1.iloc[random_1[108:135]]
# Label_0
num_0 = total_0.shape[0]
random_0 = random.sample(range(num_0), num_0)
train_0 = total_0.iloc[random_0[:400]]
test_0 = total_0.iloc[random_0[400:500]]
# train_val_1
train_val_1 = train_1.shape[0]
random_train_val = random.sample(range(train_val_1), train_val_1)
train__1 = train_1.iloc[random_train_val[:97]]
val_1 = train_1.iloc[random_train_val[97:108]]
# train_val_0
train_val_0 = train_0.shape[0]
random_train_val = random.sample(range(train_val_0), train_val_0)
train__0 = train_0.iloc[random_train_val[:360]]
val_0 = train_0.iloc[random_train_val[360:400]]
# train_val
train = pd.concat([train__1, train__0])
train = train.reset_index(drop=True)
val = pd.concat([val_1, val_0])
val = val.reset_index(drop=True)
# test
test = pd.concat([test_1, test_0])
test = test.reset_index(drop=True)

In [5]:
def toList(data):
    df = []
    for j in range(len(data)):
        index = []
        index.append(data.iloc[j]['Content'])
        index.append(data.iloc[j]['Label'])
        df.append(index)
    return df

In [6]:
train = toList(train)
val = toList(val)
test = toList(test)
test

[['還有一天好怕好怕搬家也好怕好怕活著好怕也好怕死十年了一直反覆發作我好難受新聞常常有人一下就消失了應該消失的是我才對吧...為什麼想死的人死不了不想死的人老天卻要帶走他們呢總有一天我會好，一直這樣騙自己--謝謝過的不好',
  1],
 ['把自己關在一片漆黑的房間裡 全身沒有力氣 忍受不了的痛苦只能靠哭泣和尖叫發洩夢裡跟現實好像也沒有差別 一樣像瘋子一樣發洩(只是我有力氣打人了:)) 這樣的情況真的連要結束自己都不容易我不知道悲傷從何而來 我覺得我就像一具有意識的屍體 我真希望現在立刻有人能結束我的意識--',
  1],
 ['(第一人稱視角，)媽媽跟爸爸從我出生就感情不好 所以國小時離婚了我喜歡媽媽 不喜歡爸爸但我們(姐姐)是跟爸爸住這個結果不是我能選擇的"我並不幸福 但也不可憐在那之後各種爭執..心酸除了姐姐 這個家很安',
  1],
 ['轉頭就走渣男果然是不會真心付出的我好想要去死我真的只在意我媽難怪 我姐難過我爸我不在意我現在好痛苦想跟你說我只是鬧脾氣 我還是喜歡你但來不及了而且我知道你一點也不好是我一廂情願很久沒割腕了https://i.imgur.com/cUyIwVY.jpg新的 好想吃了會死的藥喔--我被講得好難聽喔[氧氣] 請注意：s0XXX38e已婚還寄信https://www.ptt.cc/bbs/AllTogether/M.1589498591.A.B07.html全列入噁男封鎖名單：)也好我除了一開始氣頭上會回後來就算了 但還有人說我態度不值得同情...呵呵各種檢討我',
  1],
 ['我大概只能住三樓以下剛剛在9樓嘗試找窗戶開口可惜一個都沒有想像躍身的瞬間腿軟了以前住6樓常常在想但6樓應該很難-----Sent from JPTT on my iPhone--',
  1],
 ['各位晚安本版po,小魯大約1年前的生了病（類似面部痙攣的）至今仍找不到原因跟確切的治療方式，主要是因為病痛（80%）+許多平常課業上的壓力（20%）搞到我現在常常覺得非常難受，曾去做網路上的憂鬱檢測是達標的，主要狀況有沒辦法專心做別的事，常掉淚，對任何事提不起興趣，很想去死等等。 但是像我這樣很明顯知道自己痛苦的原因，而且只要身體沒發作我的心情就會好很多，但發作了就會變得想剛剛那樣負能量爆棚，是適合去身心科做檢查的嗎？請版友們給我點建議 謝謝大

In [7]:
class MyDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df #its list [['text1',label],['text2',label],...]
        self.len = len(self.df)
        self.maxlen = 250 #限制文章長度(若你記憶體夠多也可以不限)
        self.tokenizer = tokenizer  # we will use BERT tokenizer
        
    def __getitem__(self, idx):
        origin_text = self.df[idx][0]
        text_a = self.df[idx][0]
        text_b = None  #for natural language inference
        #label_tensor = None #in our case, we have label
        label_id = self.df[idx][1]
        label_tensor = torch.tensor(label_id)
        
        # 建立第一個句子的 BERT tokens
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a[:self.maxlen] + ["[SEP]"]
        len_a = len(word_pieces)
        
        if text_b is not None:
            tokens_b = self.tokenizer.tokenize(text_b)
            word_pieces += tokens_b + ["[SEP]"]
            len_b = len(word_pieces) - len_a
            
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        if text_b is None:
            segments_tensor = torch.tensor([1] * len_a,dtype=torch.long)
        elif text_b is not None:
            segments_tensor = torch.tensor([0] * len_a + [1] * len_b,dtype=torch.long)
            
        return (tokens_tensor, segments_tensor, label_tensor, origin_text)
    
    def __len__(self):
        return self.len

In [8]:
trainset = MyDataset(train, tokenizer=tokenizer)
valset = MyDataset(val, tokenizer=tokenizer)
testset = MyDataset(test, tokenizer=tokenizer)

In [9]:
# def MyDataset(Dataset):
#     data_len = len(Dataset)
#     maxlen = 250
#     text_a = Dataset['Content']
#     origin_text = Dataset['Content']
#     text_b = None  #for natural language inference
#     label_id = Dataset['Label']
#     label_tensor = torch.tensor(label_id)

#     # 建立第一個句子的 BERT tokens
#     word_pieces = []
#     for i in range(len(text_a)):
#         word_pieces += ["[CLS]"]
#         tokens_a = tokenizer.tokenize(text_a[i])
#         word_pieces += tokens_a[:maxlen] + ["[SEP]"]
#         len_a = len(word_pieces)
#         if text_b is not None:
#             tokens_b = self.tokenizer.tokenize(text_b)
#             word_pieces += tokens_b + ["[SEP]"]
#             len_b = len(word_pieces) - len_a
            
#     # 將整個 token 序列轉換成索引序列
#     ids = tokenizer.convert_tokens_to_ids(word_pieces)
#     tokens_tensor = torch.tensor(ids)
    
#     # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
#     if text_b is None:
#         segments_tensor = torch.tensor([1] * len_a,dtype=torch.long)
#     elif text_b is not None:
#         segments_tensor = torch.tensor([0] * len_a + [1] * len_b,dtype=torch.long)

#     # 將 tokens_tensor 還原成文本
#     tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
    
#     print('token:\n',tokens,'\n')
#     print('origin_text:\n',origin_text,'\n')
#     print('tokens_tensor:\n',tokens_tensor,'\n')
#     print('segment tensor:\n',segments_tensor)
#     return (tokens_tensor, segments_tensor, label_tensor, origin_text, tokens)

In [10]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
""""
create_mini_batch(samples)吃上面定義的mydataset
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""
#collate_fn: 如何將多個樣本的資料連成一個batch丟進 model
#截長補短後要限制attention只注意非pad 的部分
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 訓練集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
        
    # zero pad到該batch下最長的長度
    tokens_tensors = pad_sequence(tokens_tensors, batch_first=True)
    segments_tensors = pad_sequence(segments_tensors,batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape,dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids

# 初始化一個每次回傳 batch size 個訓練樣本的 DataLoader
# 利用 'collate_fn' 將 list of samples 合併成一個 mini-batch
BATCH_SIZE = 91
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,collate_fn=create_mini_batch,shuffle=True)
valloader = DataLoader(valset, batch_size=BATCH_SIZE,collate_fn=create_mini_batch,shuffle=False)
testloader = DataLoader(testset, batch_size=BATCH_SIZE,collate_fn=create_mini_batch,shuffle=False)

data = next(iter(trainloader))
tokens_tensors, segments_tensors, masks_tensors, label_ids = data
print(tokens_tensors)
print(segments_tensors)
print(masks_tensors)
print(label_ids)

tensor([[ 101, 5204, 4495,  ..., 5442, 8024,  102],
        [ 101, 7464, 4852,  ...,    0,    0,    0],
        [ 101, 3297, 6818,  ...,    0,    0,    0],
        ...,
        [ 101, 1184, 2407,  ..., 7076, 1164,  102],
        [ 101, 3315, 6716,  ...,    0,    0,    0],
        [ 101, 2769, 1348,  ...,    0,    0,    0]])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1,
        0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [12]:
from transformers import BertForSequenceClassification

NUM_LABELS = 2

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)


print("""
name      module
--------------------""")

for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print("{:10}{}".format(name,n) )
    else:
        print("{:10} {}".format(name, module))

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


name      module
--------------------
bert      embeddings
bert      encoder
bert      pooler
dropout    Dropout(p=0.1)
classifier Linear(in_features=768, out_features=2, bias=True)


In [13]:
%%time
from sklearn.metrics import accuracy_score

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:",device)
model = model.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
EPOCHS = 10

for epoch in range(EPOCHS):
    correct = 0
    #total = 0
    train_loss , val_loss = 0.0 , 0.0
    train_acc, val_acc = 0, 0
    n, m = 0, 0
    model.train()
    for data in trainloader:
        n += 1
        tokens_tensors, segments_tensors,masks_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)
        # outputs 的順序是 "(loss), logits, (hidden_states), (attentions)"
        
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        
        #get prediction and calulate acc
        logits = outputs[1]
        _, pred = torch.max(logits.data, 1)
        train_acc += accuracy_score(pred.cpu().tolist() , labels.cpu().tolist())

        # 紀錄當前 batch loss
        train_loss += loss.item()
    
    #validation
    with torch.no_grad():
        model.eval()
        for data in valloader:
            m += 1
            tokens_tensors, segments_tensors,masks_tensors, labels = [t.to(device) for t in data]
            val_outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)
            
            logits = val_outputs[1]
            _, pred = torch.max(logits.data, 1)
            val_acc += accuracy_score(pred.cpu().tolist() , labels.cpu().tolist())
            val_loss += val_outputs[0].item()

    print('[epoch %d] loss: %.4f, acc: %.4f, val loss: %4f, val acc: %4f' %
          (epoch+1, train_loss/n, train_acc/n, val_loss/m,  val_acc/m  ))

print('Done')

device: cuda:0


RuntimeError: cublas runtime error : the GPU program failed to execute at /opt/conda/conda-bld/pytorch_1556653215914/work/aten/src/THC/THCBlas.cu:259

In [3]:
torch.cuda.get_device_name(0)

'GeForce RTX 2060'